# Predicting marginal electricity generation from power plants in ERCOT

## Index
- [Background](#Background)  
- [Data collection](#Data-collection)  
- [Data cleaning/merging](#Data-cleaning/merging)  
    - [ERCOT data](#ERCOT-data)  
    - [EIA data](#EIA-data)  
        - [Power plant generation and fuel data from EIA-923](#Power-plant-generation-and-fuel-data-from-EIA-923)  
        - [Power plant capacity from EIA-860](#Power-plant-capacity-from-EIA-860)  
    - [EPA data](#EPA-data)  
- [Merging data for clustering](#Merging-data-for-clustering)  
- [Clustering](#Clustering)
- [Calculating hourly generation change](#Calculating-hourly-generation-change)  
- [Model training](#Model-training)

## Background
Most dispatchable electricity - electricity that can be produced on-demand - in the U.S. is generated at fossil power plants[[1]](#References). The plant, or plants, that generate more electricity in response to increased demand are called marginal generating units (MGUs). Models that predict MGUs generally fall into one of two categories: regression-based or unit-commitment economic dispatch[[2]](#References). The first category can account for effects (e.g. imperfect information) that are ignored in the second by examining real-world behavior. However, models that simply regress on historical behavior are ill-suited to making predictions about future grid conditions that differ from the data they were trained on. Applying machine learning techniques to energy sector analyses represents a pathway for potential improvements in the prediction of MGU behavior, and understanding the environmental impacts of energy transitions. This area of research is especially important as researchers and policy makers try to predict the economic and environmental impacts of vehicle electrification, demand-response management, and large scale deployment of variable renewable generating sources like wind and solar.


The goal of this project is to build a model that can predict the type of fossil MGU that will provide electricity for additional demand when given the current set of grid conditions. This type of problem can be difficult to solve, especially when the model is also trying to predict grid conditions like demand or wind generation. We are simplifying the model by treating these inputs as exogenous - the time of day or day of the year doesn't matter.

Predicting which individual power plant will provide marginal generation under a given set of grid conditions is also difficult. Individual facilities might go down for maintanence or otherwise not respond to changing grid conditions in an expected manner. We group individual fossil power plants based on their historical operating characteristics (average heat rate, capacity, capacity factor, and 95th percentile 1-hour ramp rate) using k-means clustering. The model treats each group as a single generating unit, and predicts it's change in generation given a change in grid demand.

## Data collection
Our dataset covers 9 years of operation in the ERCOT power grid, which covers most of Texas. Data sources include:

- ERCOT [[3]](#References)
    - Hourly demand for power in Megawatts (MW)
    - Hourly generation of wind power and total installed capacity (MW)
- Energy Information Agency (EIA)
    - Annual generation for each power plant (MWh) and fuels used for generation [[4]](#References)
    - Capacity of each power plant in a given year (MW) [[5]](#References)
    - Monthly natural gas prices for electric customers in Texas [[6]](#References)
    - Quarterly coal prices for electric consumers in Texas [[7]](#References)
- Environmental Protection Agency (EPA)
    - Hourly gross generation at fossil-fuel power plants larger than 25 MW [[8]](#References)

All datasets were downloaded as Excel or CSV files. All of the raw data files combined are larger than 1 GB, and can be found on the [GitHub repository](https://github.com/gschivley/ERCOT_power) for this project.

## Data cleaning/merging
Because the datasets in this project came from a number of different sources and are given on a wide range of time scales, each had to be loaded, cleaned, and put in the correct form before they could all be merged.

### ERCOT data
ERCOT hourly data on demand (load), wind generation, and wind capacity can be downloaded for a full year in a single Excel file. File extensions include both .xls and .xlsx. The 9 files did not have consistent column names across all years. Changes were made by hand in Excel.

Set up all the file paths, and column names to keep.

In [ ]:
path = 'ERCOT/Hourly wind generation'
full_xls = os.path.join(path, '*.xls')
full_xlsx = os.path.join(path, '*.xlsx')

files = glob.glob(full_xls)
files.extend(glob.glob(full_xlsx))

In [ ]:
cols = ['ERCOT Load, MW', 'Total Wind Installed, MW',
       'Total Wind Output, MW', 'Wind Output, % of Installed',
       'Wind Output, % of Load', '1-hr MW change', '1-hr % change']

Read the excel files, combine them into a single dataframe, and only keep the columns we need.

In [ ]:
ercot = pd.concat([pd.read_excel(fn, sn='numbers', index_col=0) for fn in files])
ercot = ercot.loc[:,cols]
ercot.sort_index(inplace=True)

Create a few plots to ensure the data looks good.

In [ ]:
ercot.plot(y='Total Wind Installed, MW', use_index=True)

In [ ]:
ercot.plot(y='ERCOT Load, MW', use_index=True)

### EIA data
We use EIA data for information about power plants and fuel prices.

#### Power plant generation and fuel data from EIA-923

In [ ]:
#Iterate through the directory to find all the files to import
#Modified so that it also works on macs
path = os.path.join('EIA Data', '923-No_Header')
full_path = os.path.join(path, '*.*')


eiaNames = os.listdir(path)

#Rename the keys for easier merging later
fileNameMap = {'EIA923 SCHEDULES 2_3_4_5 Final 2010.xls':2010,
                'EIA923 SCHEDULES 2_3_4_5 M Final 2009 REVISED 05252011.XLS':2009,
                'eia923December2008.xls':2008,
                'EIA923_Schedules_2_3_4_5_2011_Final_Revision.xlsx':2011,
                'EIA923_Schedules_2_3_4_5_2012_Final_Release_12.04.2013.xlsx':2012,
                'EIA923_Schedules_2_3_4_5_2013_Final_Revision.xlsx':2013,
                'EIA923_Schedules_2_3_4_5_M_12_2014_Final_Revision.xlsx':2014,
                'EIA923_Schedules_2_3_4_5_M_12_2015_Final.xlsx':2015,
                'f906920_2007.xls':2007}

#Load the files into data frames, one df per file
eiaDict = {fileNameMap[fn]:pd.read_excel(os.path.join(path, fn)) for fn in eiaNames}
eiaDict = {key:val[val["NERC Region"] == "TRE"] for key, val in eiaDict.iteritems()}

In [ ]:
#Dict of values to replace to standardize column names across all dataframes
monthDict = {"JANUARY":"JAN",
           "FEBRUARY":"FEB",
           "MARCH":"MAR",
           "APRIL":"APR",
           "MAY":"MAY",
           "JUNE":"JUN",
           "JULY":"JUL",
           "AUGUST":"AUG",
           "SEPTEMBER":"SEP",
           "OCTOBER":"OCT",
           "NOVEMBER":"NOV",
           "DECEMBER":"DEC"}
           
replaceDict = {"ELECTRIC":"ELEC",
               "&":"AND",
               "I.D.":"ID",
               "MMBTUPER":"MMBTU_PER"}
               
#Add "MMBTUMON" : "MMBTU_MON" to be replaced
for month in monthDict.values():
    replaceDict["MMBTU"+month] = "MMBTU_" + month

#Replace the column name
def rename(col):
    for old, new in monthDict.iteritems():
        col = col.replace(old, new)
        
    for old, new in replaceDict.iteritems():
        col = col.replace(old, new)
        
    col = col.replace("MMBTUS", "MMBTU")
    return col
    
#Iterate through each column name of each dataframe to standardize
for key, df in eiaDict.iteritems():
    colNames = [name.replace("\n", "_").replace(" ", "_").strip().upper() for name in df.columns]
    colNames = [rename(col) for col in colNames]
    eiaDict[key].columns = colNames

In [ ]:
#Define the columns that are necessary but are not summable
allCols = eiaDict[fileNameMap.values()[0]].columns
nonSumCols = ["PLANT_ID", "PLANT_NAME", "YEAR"]

#Define the columns that contain the year's totals (Used to calc fuel type %)
yearCols = ["TOTAL_FUEL_CONSUMPTION_QUANTITY", "ELEC_FUEL_CONSUMPTION_QUANTITY",
            "TOTAL_FUEL_CONSUMPTION_MMBTU", "ELEC_FUEL_CONSUMPTION_MMBTU",
            "NET_GENERATION_(MEGAWATTHOURS)"]


#Define the columns that are necessary and summable
sumCols = []
sumCols.extend(yearCols)
# regex = re.compile(r"^ELEC_QUANTITY_.*")
# sumCols.extend([col for col in allCols if regex.search(col)])
regex = re.compile(r"^MMBTU_PER_UNIT_.*")
sumCols.extend([col for col in allCols if regex.search(col)])
regex = re.compile(r"^TOT_MMBTU_.*")
sumCols.extend([col for col in allCols if regex.search(col)])
regex = re.compile(r"^ELEC_MMBTUS_.*")
sumCols.extend([col for col in allCols if regex.search(col)])
regex = re.compile(r"^NETGEN_.*")
sumCols.extend([col for col in allCols if regex.search(col)])

Get a list of all the different fuel type codes.

In [ ]:
fuelTypes = []
fuelTypes.extend([fuelType for df in eiaDict.values() for fuelType in df["REPORTED_FUEL_TYPE_CODE"].tolist()])
fuelTypes = set(fuelTypes)

EIA-923 splits out electricity generated at a power plant generating units and the type of fuel. We are aggregating everything to the facility level and assigning a single fuel type to the plant. This fuel type is used to filter out non-fossil plants and for analysis of what types of power plants are in the clusters.

Three parts to aggregate by facility, and to calculate the % of each type of fuel. This will take a few minutes to run.  

The end result is aggEIADict.

In [ ]:
#Actually calculate the % fuel type for each facility grouping
def calcPerc(group, aggGroup, fuelType, col):
    #Check to see if the facility has a record for the fuel type, and if the total column > 0
    if len(group[group["REPORTED_FUEL_TYPE_CODE"] == fuelType]) > 0 and aggGroup[col] > 0:
        #summing fuel type because a facility may have multiple plants with the same fuel type        
        return float((group[group["REPORTED_FUEL_TYPE_CODE"] == fuelType][col]).sum())/aggGroup[col] 
    else:
        return 0

#Perform the aggregation on facility level
def aggAndCalcPerc(group):
    aggGroup = group.iloc[0][nonSumCols] #Get the non-agg columns
    aggGroup = aggGroup.append(group[sumCols].sum())   #Aggregate the agg columns and append to non-agg
    percCols = {col + " %" + fuelType:calcPerc(group, aggGroup, fuelType, col) for col in yearCols for fuelType in fuelTypes}
    aggGroup = aggGroup.append(pd.Series(percCols))
    return aggGroup    

#Iterate through each dataframe to perform aggregation by facility
aggEIADict = dict()
for key, df in eiaDict.iteritems():
    gb = df.groupby(by="PLANT_ID")
    #aggGroup will be a list of panda series, each series representing a facility
    aggGroup = [aggAndCalcPerc(gb.get_group(group)) for group in gb.groups]
    aggEIADict[key] = pd.DataFrame(aggGroup)

**Combine all df's from the dict into one df**

Concat all dataframes, reset the index, determine the primary fuel type for each facility, filter to only include fossil power plants, and export as a csv

In [ ]:
all923 = pd.concat(aggEIADict)
all923.reset_index(drop=True, inplace=True)

In [ ]:
def top_fuel(row):
    #Fraction of largest fuel for electric heat input 
    try:
        fuel = row.iloc[1:27].idxmax()[29:]
    except:
        return None
    return fuel

In [ ]:
all923['FUEL'] = all923.apply(top_fuel, axis=1)

Because the EPA data only includes power plants that burn fossil fuels, we are only keeping these facilities. The codes below correspond to:
- Diesel fuel oil
- Lignite coal
- Natural gas
- Petroleum coke
- Subbituminous coal

In [ ]:
fossil923 = all923.loc[all923['FUEL'].isin(['DFO', 'LIG', 'NG', 'PC', 'SUB'])]

#### Power plant capacity from EIA-860

In [ ]:
# Iterate through the directory to find all the files to import
path = os.path.join('EIA Data', '860-No_Header')
full_path = os.path.join(path, '*.*')

eia860Names = os.listdir(path)

# Rename the keys for easier merging later
fileName860Map = {  'GenY07.xls':2007,
                    'GenY08.xls':2008,
                    'GeneratorY09.xls':2009,
                    'GeneratorsY2010.xls':2010,
                    'GeneratorY2011.xlsx':2011,
                    'GeneratorY2012.xlsx':2012,
                    '3_1_Generator_Y2013.xlsx':2013,
                    '3_1_Generator_Y2014.xlsx':2014,
                    '3_1_Generator_Y2015.xlsx':2015}

#Load the files into data frames, one df per file
eia860Dict = {fileName860Map[fn]:pd.read_excel(os.path.join(path, fn)) for fn in eia860Names}  

In [ ]:
#Dict of values to replace to standardize column names across all dataframes
renameDict = {  "PLNTCODE":"PLANT_ID",
                "PLANT_CODE":"PLANT_ID",
                "Plant Code":"PLANT_ID",
                "NAMEPLATE":"NAMEPLATE_CAPACITY(MW)",
                "Nameplate Capacity (MW)":"NAMEPLATE_CAPACITY(MW)"}

#Replace the column name
def rename860(col):
    for old, new in renameDict.iteritems():
        col = col.replace(old, new)
    return col

#Iterate through each column name of each dataframe to standardize and select columns 'PLANT_ID', 'NAMEPLATE_CAPACITY(MW)'
for key, df in eia860Dict.iteritems():
    colNames = [rename860(col) for col in df.columns]
    eia860Dict[key].columns = colNames
    eia860Dict[key] = eia860Dict[key][["PLANT_ID", "NAMEPLATE_CAPACITY(MW)"]]

Power plants can include multiple generating units. We are aggregating the total generating capacity of units in the EIA-860 database to the facility level.

In [ ]:
# Iterate through each dataframe to perform aggregation by PLANT_ID
for key, df in eia860Dict.iteritems():
    gb = df.groupby(by='PLANT_ID').apply(lambda x: x['NAMEPLATE_CAPACITY(MW)'].sum())
    eia860Dict[key]['NAMEPLATE_CAPACITY(MW)'] = eia860Dict[key].PLANT_ID.apply(gb.get_value)
    eia860Dict[key] = eia860Dict[key].drop_duplicates(subset=['PLANT_ID', 'NAMEPLATE_CAPACITY(MW)'])
    eia860Dict[key] = eia860Dict[key].sort_values(by='PLANT_ID').reset_index(drop=True)

#### Fuel price data
**Need to decide what level of merging to include in this section.**

Start with natural gas price

In [ ]:
path = os.path.join('EIA data', 'Natural gas price', 'N3045TX3m.xls')
ng_price = pd.read_excel(path, sheetname='Data 1', header=2)

In [ ]:
ng_price.rename(columns={"Texas Natural Gas Price Sold to Electric Power Consumers (Dollars per Thousand Cubic Feet)":
                         "NG Price ($/mcf)"}, inplace=True)

In [ ]:
ng_price.loc[:,'Month'] = ng_price.loc[:,'Date'].apply(lambda x: x.month)
ng_price.loc[:,'Year'] = ng_price.loc[:,'Date'].apply(lambda x: x.year)

Coal data

In [ ]:
path = os.path.join('EIA data', 'Coal prices', 'Texas electric sector coal price.xlsx')
coal_temp = pd.read_excel(path, sheetname='Sheet1')

### EPA data
EPA hourly data is stored in a series of .txt files that each cover a 6 month period.

In [ ]:
#Read the EPA files into a dataframe
path2 = os.path.join('EPA air markets')
epaNames = os.listdir(path2)
filePaths = {dn:os.path.join(path2, dn, "*.txt") for dn in epaNames}
filePaths = {dn:glob.glob(val) for dn, val in filePaths.iteritems()}
epaDict = {key:pd.read_csv(fp, index_col = False) for key, val in filePaths.iteritems() for fp in val}

The DataFrames in epaDict contain all power plants in Texas. We can filter on NERC REGION so that it only includes ERCOT.

In [ ]:
#Rename the column names to remove the leading space.
for key, df in epaDict.iteritems():
    colNames = [name.upper().strip() for name in df.columns]
    colNames[colNames.index("FACILITY ID (ORISPL)")] = "PLANT_ID"
    epaDict[key].columns = colNames
    
#Convert DATE to datetime object
#Add new column DATETIME with both date and hour
for key, df in epaDict.iteritems():
    epaDict[key]["DATE"] = pd.to_datetime(df["DATE"])
    epaDict[key]['DATETIME'] = df['DATE'] + pd.to_timedelta(df['HOUR'], unit='h')

In [ ]:
#Boolean filter to only keep ERCOT plants
for key, df in epaDict.iteritems():
    epaDict[key] = df[df["NERC REGION"] == "ERCOT"].reset_index(drop = True)

### 

### 

## Merging data for clustering

In [ ]:
clusterDict = dict()
for key, df in eia860Dict.iteritems():
    clusterDict[key] = pd.merge(aggEIADict[key], eia860Dict[key], how='left', on='PLANT_ID')[['PLANT_ID', 'NAMEPLATE_CAPACITY(MW)']]
    clusterDict[key].rename(columns={'NAMEPLATE_CAPACITY(MW)': 'capacity', 'PLANT_ID': 'plant_id'}, inplace=True)

In [ ]:
def top_fuel(row):
    #Fraction of largest fuel for electric heat input 
    try:
        fuel = row.idxmax()[29:]
    except:
        return None
    return fuel

Calculate Capacity factor, Efficiency, Fuel type

In [ ]:
for key, df in clusterDict.iteritems():
    clusterDict[key]['year'] = key
    clusterDict[key]['capacity_factor'] = aggEIADict[key]['NET_GENERATION_(MEGAWATTHOURS)'] / (8670*clusterDict[key]['capacity'])
    clusterDict[key]['efficiency'] = (aggEIADict[key]['NET_GENERATION_(MEGAWATTHOURS)']*3.412)/(1.0*aggEIADict[key]['ELEC_FUEL_CONSUMPTION_MMBTU'])
    clusterDict[key]['fuel_type'] = aggEIADict[key][fuel_cols].apply(top_fuel, axis=1)
    clusterDict[key] = clusterDict[key][clusterDict[key]['fuel_type'].isin(['SUB', 
                                                                            'LIG', 
                                                                            'DFO',
                                                                            'NG', 
                                                                            'PC'])]

Merge all EPA files in one df

In [ ]:
columns = ['PLANT_ID', 'YEAR', 'DATE', 'HOUR', 'GROSS LOAD (MW)']
counter = 0
for key, df in epaDict.iteritems():
    if counter == 0:
        result = epaDict[key][columns]
        counter = 1
    else:
        result = result.append(epaDict[key][columns], ignore_index=True)

Function to calculate the ramp rate for every hour

In [ ]:
def plant_gen_delta(df):
    """
    For every plant in the input df, calculate the change in gross load (MW)
    from the previous hour.
    
    input:
        df: dataframe of EPA clean air markets data
    return:
        df: concatanated list of dataframes
    """
    df_list = []
    for plant in df['PLANT_ID'].unique():
        temp = df.loc[df['PLANT_ID'] == plant,:]
        gen_change = temp.loc[:,'GROSS LOAD (MW)'].values - temp.loc[:,'GROSS LOAD (MW)'].shift(1).values
        temp.loc[:,'Gen Change'] = gen_change
        df_list.append(temp)
    return pd.concat(df_list)

In [ ]:
ramp_df = plant_gen_delta(result)

Calculate the 95th percentile ramp rate for each plant in each year. We use the 95th percentile to avoid data errors.

In [ ]:
cols = ['PLANT_ID', 'YEAR', 'Gen Change']

ramp_rate_list = []
for year in ramp_df['YEAR'].unique():
    for plant in ramp_df.loc[ramp_df['YEAR']==year,'PLANT_ID'].unique():
        # 95th percentile ramp rate per plant per year
        ramp_95 = ramp_df.loc[(ramp_df['PLANT_ID']== plant) & 
                              (ramp_df['YEAR']==year),'Gen Change'].quantile(0.95, interpolation='nearest')
        ramp_rate_list.append([plant, year, ramp_95])

Merge the EIA data (clusterDict) with the ramp rate data

In [ ]:
for key, df in clusterDict.iteritems():
    clusterDict[key] = pd.merge(clusterDict[key], ramp_rate_df, how='left', on=['plant_id', 'year'])

## Clustering

This section uses the class `Clusters`, which is defined in `cluster.py`.

In [ ]:
from cluster import Clusters

Right now `Clusters` reads the data in from a file.

In [ ]:
filename = 'Cluster_Data_2.csv'
path = '../Clean Data'
fullpath = os.path.join(path, filename)
cluster = Clusters(fullpath)

The `make_clusters()` function uses KMeans from scikit-learn to produce clusters across a range of k-values.

In [ ]:
cluster.make_clusters(n_clusters=range(4,26))

`evaluate_clusters()` calculates and plots the calinski-harabaz score and silhouette score for each k value

In [ ]:
cluster.evaluate_clusters()

From the figures above, it is difficult to determine an optimal number of clusters. The silhouette score clearly shows that we need more than 5 clusters. 6 looks like a good number, but we also look at 12 to see if it will provide a better model.

In [ ]:
labeled_plants = cluster.label_and_export(k=6)

## Calculating hourly generation change

## Model training

## References
1. Siler-Evans, K., Azevedo, I. L., & Morgan, M. G. (2012). Marginal Emissions Factors for the U.S. Electricity System. Environmental Science & Technology, 46(9), 4742–4748.
2. Graff Zivin, J. S., Kotchen, M. J. & Mansur, E. T. Spatial and temporal heterogeneity of marginal emissions: Implications for electric cars and other electricity-shifting policies. Journal of Economic Behavior & Organization 107, Part A, 248–268 (2014).
3. http://www.ercot.com/gridinfo/generation
4. https://www.eia.gov/electricity/data/eia923/
5. https://www.eia.gov/electricity/data/eia860/
6. http://www.eia.gov/dnav/ng/hist/n3045tx3m.htm
7. [EIA Coal Data Browser](http://www.eia.gov/beta/coal/data/browser/#/topic/45?agg=1,0&geo=vvvvvvvvvvvvo&rank=l&linechart=~~COAL.SHIP_PLANT_PRICE.TX-LIG.Q~COAL.SHIP_PLANT_PRICE.TX-SUB.Q&columnchart=COAL.SHIP_PLANT_PRICE.US-TOT.Q&map=COAL.SHIP_PLANT_PRICE.US-TOT.Q&freq=Q&start=200801&end=201601&chartindexed=0&ctype=linechart&ltype=pin&rtype=s&maptype=0&rse=0&pin=)
8. https://ampd.epa.gov/ampd/